<img src="Imatges/PortadaGeneral.jpg" width="1200" />

# 1 INTRODUCTION

## 1.1 Summary
In this project I’m going to answer one question:
* Is there a relationship between the socioeconomic status of people living in a district and the profile of venues in it?
The target audience are:
* Companies wishing to stablish in Girona or new businesses that want to open in Girona
* Local city council

The project has two main parts:
* Data acquisition and data preparation.
  * Fetch geographical information, process it and feed it to the Foursquare queries.
  * Process the results of Foursquare queries
  * Fetch and process statistical data 
* From data to information.
  * Apply machine learning methodologies and techniques to extract relevant information out of data
  * Data visualization will help on the understanding of the insights obtained so far.

## 1.2 Background
Cities, as large human settlements, flourished in parallel with the development of agriculture and have been characterized by the specialization and social division of labor. Great cities have existed in all civilizations of the human history and they have been planned and managed by bureaucracy systems which needed data science to do the job. One example that I like to use to link cities and data science is the development of the concept of the number zero. Even if in some of them the number was not invented, most of the ancient civilizations developed the concept of zero and used it for calculations in many areas like accounting, astronomy, geography etc.

Cities are great places to enhance human interaction both for the good and for the bad. In both cases data science is paramount in improving human life. In this project I develop a process to gather data from cities which can be used by different stakeholders. I’m mainly targeting
* Companies which want to explore the socioeconomic landscape of cities to make informed decisions about their businesses
* Government and governmental agencies seeking to improve resource allocation, from the deployment of police forces to budget allocation
In all these cases, a fine granulation of the city would be of major interest as it allows us to make a zoom to small patches that can be also aggregated conveniently. Often, postal code areas are used to do this division of the city, but I think that census areas are a better choice. Census areas in Catalonia are areas where there are between 500 and 2000 electors. One census area cannot include territories of more than one municipality, and they can be grouped administratively into census districts. Moreover, official statistical data is geographically segmented by census districts.

In this project, I use Girona as a case study that can be later extrapolated to other cities in the country. Girona will be divided in census areas and from this reference area, Foursquare will be used to locate venues in neighborhoods.

### 1.2.1 Mapping venues
Mapping venues in towns is the first step in the process of typifying census areas with relation to the profile of facilities and commercial activities. This profiling can be complemented and/or further correlated with other statistical data like socioeconomic status of inhabitants.

For example, if an area is characterized by the presence of restaurants and small households occupied mainly by young people, a real state company can focus its efforts in selling or hiring in this area to this profile of young single or couple wealthy people. If there is another area with parks and supermarkets and households are of an average four people, this means that it may be a family neighborhood. A supermarket chain may consider it as a target neighborhood.

For governmental stakeholders, checking the reviews of businesses around an area might be a very useful source of information to plan specific support actions for this or that particular area.


# 2 DATA

Though we live in times of data hype and terabytes of data accumulate endlessly, data is often kept at very different places, in different formats. Fortunately, there are official governmental sources of data which are reliable. These sources are not completely coherent in terms of format, but they are manageable.
In this project I’m going to use three main sources of data:
* Data from the Cartographic Institute of Catalonia [ICC](https://www.icgc.cat/Administracio-i-empresa/Descarregues/Capes-de-geoinformacio/Seccions-censals)
* Data from Girona’s local council that can be obtained either from Girona Open Data [GOD](https://www.girona.cat/opendata/) or from l’[Observatori](https://terra.girona.cat/apps/observatori/).
* Data from [Foursquare](https://foursquare.com/)

# 3 METHODOLOGY

## 3.1 Prepping data

### 3.1.1 Geographic data 
<img src="Imatges/DataProcessing_small.jpg"  style="float:right" />
The data can be downloaded directly from the Jupiter notebook, or can be downloaded externally and kept in a local file to be further imported in the notebook. I chose the second system to avoid that network problems interfere with the development process.

In the figure beside, there is a schematic representation of the transformations that have been performed in the Geografic data from ICC and GOD. In the original sources, geographical data is kept in Shapefile format which is a common format used in geographical information systems (GIS). However, the geojson format is increasingly used in data science and is the format that is used to feed map tools like Folium and Choropleth. Thus, the first step will be to transform .shp files into .json files thanks to a library PyShp. I will define the function *shpToGeoJSON* were I will use the [PyShp](https://pypi.org/project/pyshp/) tools to read .shp files and return the information in the form of python dictionaries.
In both ICC and GOD the geographical data is kept in utm coordinates, but many current applications are better fed with WGS84 coordinates. So, I will transform the utm coordinates to WGS84 coordinates with the help of a library [PyProj](https://pypi.org/project/pyproject/). For this job I’ll create the function *getWGS84Coordinates*.

Once the data is in json format and the coordinates are coded with the WGS84 system, I’ll make some small transformations like changing some town codes by the official name of the town, this is for the sake of readability of data when observing them. I’ll also create a merged code joining the district code and the section code to uniquely identify each census section in Girona.

In [ ]:
#!pip install PyShp #This is a libary to read and manipulate .shp files
#!pip install pyproj #This is a library for performing coordinate transformations

In [96]:
from datetime import date #library to get current date and time
import folium #Library to draw maps
import json #To manipulate json files, ie. transforming json to dict and viceversa
import math # Library with mathematical functions
import numpy as np # library to handle data in a vectorized manner
import os # library to operate with the file system
import pandas as pd # library for data analsysis
from pyproj import Proj #This is a library for performing coordinate transformations
import requests # library to handle requests
from scipy.spatial import ConvexHull, convex_hull_plot_2d # Library to do geometric calculations
import shapefile #This is a libary to read and manipulate .shp files
from shapely.geometry import Point # Library to do geometric calculations
from shapely.geometry.polygon import Polygon # Library to do geometric calculations #Library to perform cluster analysis
from sklearn.cluster import KMeans #Library to perform cluster analysis
import sys # Useful library for showing errors
import time # Library to manage time and timestamps

#### Function to transform coordinates to WGS84 in geo_JSON dictionaries

In [2]:
def getWGS84Coordinates(transformationParameters, JSON):
    myProj = Proj(projectParameters)
    list0 = []
    for coordinate in range(0, len(JSON['coordinates'][0])):
        east_utm = JSON['coordinates'][0][coordinate][0]
        north_utm = JSON['coordinates'][0][coordinate][1]
        lon, lat = myProj(east_utm, north_utm, inverse=True)
        list0.append([lon, lat]) #beware that Choropleth takes longitude first
    JSON['coordinates'] = [list0]
    return JSON

#### Function to transform .shp data to geoJSON and save it to file**

In [4]:
# This function is to load a .shp file transform it to a json object and then save its contents as a .geojson file
def shpToGeoJSON(PathIn, PathOut, Name, Encoding, CoordinateTransformParameters):
    inPath = PathIn + Name + '.shp'
    outPath = PathOut + Name + '.geoJSON'
    import shapefile 
    reader = shapefile.Reader(inPath, encoding = Encoding)
    fields = reader.fields[1:]
    field_names = [field[0] for field in fields]
    buffer = []
    for sr in reader.shapeRecords():
        atr = dict(zip(field_names, sr.record))
        geom = sr.shape.__geo_interface__
        #After a lot of strugle I realized that the best way to modify the geoJSON coordinates is to transform them here before saving as geoJSON
        geom = getWGS84Coordinates(CoordinateTransformParameters, geom)
        buffer.append(dict(type="Feature", \
        properties=atr, geometry=geom )) 
    # write the GeoJSON file
    from json import dumps
    import codecs
    geojson = codecs.open(outPath, "w", "utf-8")
    geojson.write(dumps({"type": "FeatureCollection",\
    "features": buffer}, indent=2) + "\n")
    geojson.close()

#### Transformation of the .shp files with utm coordinates into .json files with WGS84 coordinates

In [5]:
# Here we use the function shpToGeoJSON to transform several .shp files and save the content in .geoJSON files
PathIn = "C:/Users/user/Documents/Els_meus_documents/projectes/CompetitiveIntelligence/Data/Seccions censals/"
PathOut = "C:/Users/user/Documents/Els_meus_documents/projectes/CompetitiveIntelligence/Data/geoJSON/"
Names = ['Catalunya2016', 'Girona', 'GironaCentre']

In [6]:
# Here we use the function shpToGeoJSON to transform several .shp files and save the content in .geoJSON files
PathIn = "C:/Users/user/Documents/Els_meus_documents/projectes/CompetitiveIntelligence/Data/Seccions censals/"
PathOut = "C:/Users/user/Documents/Els_meus_documents/projectes/CompetitiveIntelligence/Data/geoJSON/"
Names = ['Catalunya2016', 'Girona', 'GironaCentre']
#Here we define the parameters to fetch coordinates with PyProject library
projectParameters = "+proj=utm +zone=31N +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs"

for Name in Names:
    if "Centre" in Name:
        Encoding = 'latin-1'
    else:
        Encoding = 'utf-8'
    shpToGeoJSON(PathIn, PathOut, Name, Encoding, projectParameters)

#### Upload of the data in json format from the .geojson files**

In [20]:
Names = ['Catalunya2016', 'Girona', 'GironaCentre']
PathOut = "C:/Users/user/Documents/Els_meus_documents/projectes/CompetitiveIntelligence/Data/geoJSON/"
for Name in Names:
    Path = PathOut + Name + '.geoJSON'
    with open(Path, 'r') as f:
        if 'Catalunya' in Name:
            Catalunya = json.loads(f.read())
        elif 'Centre' in Name:
            Centre = json.loads(f.read())
        else:
            Girona = json.loads(f.read())

#### Changing the city code by city names
In the catalan database of census sections, cities are coded numerically. In order to make it more human readable we change the codes by the official names. This is a two step process.
1. We build a dictionary with codes and names
2. we loop trough the geo_JSON dict to change numeric codes by names

#### Build a dictionary with key: city code, and value: name of the city

In [21]:
nomenclator = pd.read_csv('C:/Users/user/Documents/Els_meus_documents/projectes/CompetitiveIntelligence/Data/nomenclatorPrincipatCatalunya.csv')
nomenclator_dict = {}
for x in range(0, len(nomenclator)):
   if nomenclator.iloc[x][0] == 'Municipi':
        if len(str(nomenclator.iloc[x][1])) == 5:
            codi = '0' + str(nomenclator.iloc[x][1])
        else:
            codi = str(nomenclator.iloc[x][1])
        nomenclator_dict[codi] = nomenclator.iloc[x][6]

#### Function to change the numeric municipal codes by city names
We use the data from the official Catalan Institute of Statistics [IDESCAT](https://www.idescat.cat/codis/?cin=0&nom=&ambit=a&cic=0&codi=&pobi=&pobf=&id=50&n=22&inf=c&t=01-01-2019)

In [22]:
def changeTownCodeByName(JSON, DICT):
    if 'MUNICIPI' in JSON['features'][0]['properties'].keys():
        for feature in range(0, len(JSON['features'])):
            code = JSON['features'][feature]['properties']['MUNICIPI']
            JSON['features'][feature]['properties']['MUNICIPI'] = DICT[code]
    if 'SECCIÓ' in JSON['features'][0]['properties'].keys():
        for feature in range(0, len(JSON['features'])):
            JSON['features'][feature]['properties']['SECCIO'] = JSON['features'][feature]['properties']['SECCIÓ']
            del JSON['features'][feature]['properties']['SECCIÓ']
    else:
        print("No hi ha codi de municipi")
    return JSON
    

#### Transforming the original data
Applying the functions to improve datasets

In [23]:
dataSets = [Catalunya, Girona, Centre]

j = 0
for i in dataSets:
    j +=1
    try:
        i = changeTownCodeByName(i, nomenclator_dict)
    except ValueError:
         print("Got a problem in", j)
    except:
        print("Unexpected error:", sys.exc_info()[0])
        raise

No hi ha codi de municipi
No hi ha codi de municipi


<img src="Imatges/geojson.png"  style="float:right" />
The final .geojson dictinaries look like in the figures beside. I have highlighted in yellow the two main keys of the dictionary, "Properties" which associated value is a dictionary with general information, and "Geometries" which associated value is a dictionary that contains the coordinates of the polygons (highlighted in red).

The geojson from the ICC contains the data for all the census sections of Catalonia. We are not going to use this data in this report. But it is a key database to expand the model to the rest of the country.
The geojson containing the coordinates of the census sections of Girona will be used to draw the sections in the map and to make the calls to Foursquare.

The last geojson, containing the coordinates of the census districts of Girona will be used to overlay the district on the map with the sections. It will also be very important to draw statistical information on the map.


### 3.1.2 Foursquare data
Foursquare is a powerful database which that can be easily accessed with APIs provided in its developer page. However, there are some limitations that forces us to do some extra work. The main limitation I found is that the results of searches for venues are given in circle area defined by a radius. I want to assign each venue to the census section to where it belongs to. In order to achieve that I will make the following steps:
 1.	Determine the centroid of each census are which are, for the most, quite irregular polygons. For this I use the library *scipy.spatial* which has a tool to calculate the convex hull of a polygon. Then the centroid of the convex hull is the mean of the x coordinates and the mean of the y coordinates of the hull. I used this approximation because it gives better results than the simpler approximation of determining the centroid as the means of the points of the raw polygon (data not shown).
 2.	Once the centroid is found I use the function *distanceBetweenPoints* to determine the longest distance between the centroid and all the vertices. This will be the distance used to assign the parameter RADIUS in the Foursquare search.
 3.	Next step is to retrieve Foursquare data using latitude and longitude of the centroid of each census section and the radius as the longest distance between centroid and vertices. But this will yield a lot of overlapping information as the circle defined by the radius in each census section will intersect with the circles of the neighboring sections. 
 4.	Once venue data are obtained, venues that do not belong to the census section must be filtered out. In order to do this, I’ll have to check if venue coordinates are inside the polygon which defines the census section. This can be easily done using the function *polygon.contains* of the [Shapely](https://pypi.org/project/Shapely/) library that I’ll execute inside the local function *ifInDistrict*.
 5.	The final step is to convert all data in a Panda’s data frame.
 

#### Determining the coordinates of the centroid and the max radius of every census district
The following three functions are for:
* Calculate the distance between two points given their coordinates (distanceBetweenPoints)
* Generate a census code by joining the census district code with the census section code (codiSeccioCensal)
* Calculate the centroid and the maximum radius of a polygon (calcCentroidOfPolygon)

In [55]:
def distanceBetweenPoints(lat1, lon1, lat2, lon2):
    m_per_deg_lat = 111.132954
    m_per_deg_lon = 40075 * math.cos(lat1 ) / 360
    dist = math.sqrt(((lat1 - lat2) * m_per_deg_lat)**2 + ((lon1 - lon2) * m_per_deg_lon)**2)
    return dist

In [56]:
def codiSeccioCensal(districte, seccio):
    if len(str(districte)) < 2:
        codiDistricte = '0' + str(districte)
    else:
        codiDistricte = '0' + str(districte)
    if len(str(seccio)) < 2:
        codiSeccio = '00' + str(seccio)
    elif len(str(seccio)) < 3:
        codiSeccio = '0' + str(seccio)
    else:
        codiSeccio = str(seccio)
    return codiDistricte + codiSeccio

In [57]:
# There are different ways to approach the centroid of a polygon. The Simplest one is to take the x and y of the centroid
# as the average of the x an y of the vertices. This gives not optimal result for complex polygones. An alternative is to 
# use a convex hull and then calculate the centroid of the convex hull
def calcCentroidOfPolygon(JSON):
    from scipy.spatial import ConvexHull, convex_hull_plot_2d
    Centroid = []
    for feature in range(0, len(JSON['features'])):
        x = JSON['features'][feature]['geometry']['coordinates'][0]
        points = np.asarray(x, dtype = np.float32)
        hull = ConvexHull(points)
        centroid = [np.mean(points[hull.vertices,1]), np.mean(points[hull.vertices,0])]
        dist = []
        for point in range(0, len(hull.vertices)):
            dist.append(distanceBetweenPoints(centroid[0], centroid[1], points[hull.vertices[point],1], points[hull.vertices[point],0]))
        SeccioCensal = codiSeccioCensal(JSON['features'][feature]['properties']['DISTRICTE'], JSON['features'][feature]['properties']['SECCIO'])
        Centroid.append( [SeccioCensal,centroid[0], centroid[1], round(max(dist) * 1000, 0),x])
    return Centroid
    

In [58]:
Centroid = calcCentroidOfPolygon(Girona)

Centroid is a list of lists. Each of the lists contains the following items:
1. The code of the census section
2. The latitude of the centroid of the census section
3. The longitude of the centroid
4. The maximum radius of the polygon as the longest distance between the centroid and the vertices.
5. A list with the coordinates of the vertices of the census section

#### Adding data from places using Foursquare database

##### Fetching the venues in census districts
Now we know the centroid of each census district and we know the largest distance in meters between between the centroid and the vertices. Now we are going to:
1. get all venues around each centroid
2. check whether the venues are within the district or not
3. if it is in the district we keep the information

First we define the function to fetch venues around a point. The output is a data frame with the basic nformation of each venue

In [59]:
def ifInDistrict(Lon, Lat, Poly):
    point = Point(Lon, Lat)
    polygon = Polygon(Poly)
    return polygon.contains(point)


In [60]:
def getNearbyVenues(Names, Latitudes, Longitudes, Radius, Poly):
    from shapely.geometry import Point
    from shapely.geometry.polygon import Polygon
    venues_list=[]
    venues = {}
    for name, lat, lng, rad, poly in zip(Names, Latitudes, Longitudes, Radius, Poly):
        results = {}
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            rad, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # Check if venue is inside district
        i = 0
        if results:
            for v in results:
                if v['venue']['name'] in venues:
                    pass
                elif ifInDistrict(v['venue']['location']['lng'], v['venue']['location']['lat'], poly):
                    i +=1
                    venues_list.append([name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']])
                    venues[v['venue']['name']] = 0
                else:
                    pass
            else:
                pass
        if i >= 100:
            print("Beware, in census district", name, "there are more than 100 venues")
        time.sleep(0.5)
    nearby_venues = pd.DataFrame([item for item in venues_list])
    nearby_venues.columns = ['Census section', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now, we loop through the census districts to get the venues in them

In [61]:
CLIENT_ID = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'YYYYYYYYYYYYYYYYYYYYYYYYYYYYYY' # your Foursquare Secret
VERSION = '20200330' # Foursquare API version
LIMIT = 100
Nam =[]
Lat = []
Lon = []
Rad = []
Pol = []
Nam = [x[0] for x in Centroid]
for x in Centroid:
    Nam.append(x[0])
    Lat.append(x[1])
    Lon.append(x[2])
    Rad.append(x[3])
    Pol.append(x[4])
GironaVenues = getNearbyVenues(Nam, Lat, Lon, Rad, Pol)


In [69]:
GironaVenues.head(10)


,Census section,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Census district
0,"""02012""",41.975842,2.819030,Sweets by Abraham Balaguer,41.976007,2.819248,Dessert Shop,02
1,"""02012""",41.975842,2.819030,Cafeteria Tornés,41.975807,2.818575,Café,02
2,"""03001""",41.983753,2.821557,Umai,41.983094,2.821719,Japanese Restaurant,03
3,"""03001""",41.983753,2.821557,Museu del Cinema,41.983756,2.822213,Museum,03
4,"""03001""",41.983753,2.821557,Llibreria 22,41.984379,2.822301,Bookstore,03
5,"""03001""",41.983753,2.821557,Plaça de Josep Pla,41.983295,2.821843,Plaza,03
6,"""03001""",41.983753,2.821557,Plaça de la Constitució,41.983750,2.821248,Plaza,03
7,"""03001""",41.983753,2.821557,Bionèctar,41.984095,2.821569,Vegetarian / Vegan Restaurant,03
8,"""03001""",41.983753,2.821557,Restaurant Gran Muralla,41.984567,2.821547,Chinese Restaurant,03
9,"""03001""",41.983753,2.821557,Apple Cafè Girona,41.982338,2.822440,Sushi Restaurant,03


In [72]:
today = str(date.today())
filename = "C:/Users/user/Documents/Els_meus_documents/projectes/CompetitiveIntelligence/Data/Foursquare/" + "GironaVenues" + today + ".csv"
GironaVenues.to_csv(filename, index=False)

In [78]:
# filename = "C:/Users/user/Documents/Els_meus_documents/projectes/CompetitiveIntelligence/Data/Foursquare/GironaVenues2020-04-25.csv"
GironaVenues = pd.read_csv(filename)  
GironaVenues['Census section']= GironaVenues['Census section'].str.slice(1, 6, 1) 
from csv import reader
districtes = {}
with open(csv_file, 'r') as f:
    csv_reader = reader(f)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        districtes['0' + row[0]] = row[1]

GironaVenues['Neighborhood'] = [districtes[x] for x in GironaVenues['Census section']]
GironaVenues.head()

,Census section,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood
0,02012,41.975842,2.819030,Sweets by Abraham Balaguer,41.976007,2.819248,Dessert Shop,Eixample
1,02012,41.975842,2.819030,Cafeteria Tornés,41.975807,2.818575,Café,Eixample
2,03001,41.983753,2.821557,Umai,41.983094,2.821719,Japanese Restaurant,Centre
3,03001,41.983753,2.821557,Museu del Cinema,41.983756,2.822213,Museum,Centre
4,03001,41.983753,2.821557,Llibreria 22,41.984379,2.822301,Bookstore,Centre


### 3.1.3 Clustering analysis
#### 3.1.3.1 Clustering by Venues
Here we are going to prepare the foursquare data to make a clustering analysis on neighborhood (census district)

#### Analyze each neighborhood

In [81]:
# one hot encoding
Girona_onehot = pd.get_dummies(GironaVenues[['Venue Category']], prefix="", prefix_sep="")

# add Postal Code column back to dataframe
Girona_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
Girona_onehot.insert(0, 'Neighborhood', GironaVenues['Neighborhood'])  

Girona_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Argentinian Restaurant,Art Museum,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,...,Tennis Court,Theater,Toy / Game Store,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Store,Volleyball Court,Wine Bar,Wine Shop
0,Eixample,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Eixample,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Centre,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Centre,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Centre,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Grouping rows by census district and taking the mean of the frequency of occurrence of each category

In [82]:
Girona_grouped = Girona_onehot.groupby('Neighborhood').mean().reset_index()
Girona_grouped

,Neighborhood,African Restaurant,American Restaurant,Argentinian Restaurant,Art Museum,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,...,Tennis Court,Theater,Toy / Game Store,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Store,Volleyball Court,Wine Bar,Wine Shop
0,Centre,0.010417,0.000000,0.010417,0.020833,0.000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.020833,0.010417,0.000000,0.010417,0.010417,0.000000,0.000000,0.03125,0.000000
1,Eixample,0.000000,0.010101,0.000000,0.000000,0.000,0.020202,0.010101,0.010101,0.0,...,0.000000,0.000000,0.000000,0.020202,0.000000,0.020202,0.000000,0.000000,0.00000,0.010101
2,Est,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
3,Mas Xirgu,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
4,Montjuïc,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
5,Nord,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.125000
6,Oest,0.000000,0.000000,0.000000,0.000000,0.000,0.040000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
7,Santa Eugènia,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.00000,0.000000
8,Sud,0.032258,0.000000,0.000000,0.000000,0.000,0.000000,0.064516,0.000000,0.0,...,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000


#### Grouping the 10 top venues in each neighborhood

In [84]:
# first we write a function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
#Now, we create a new data frame
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
districts_venues_sorted = pd.DataFrame(columns=columns)
districts_venues_sorted['Neighborhood'] = Girona_grouped['Neighborhood']

for ind in np.arange(Girona_grouped.shape[0]):
    districts_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Girona_grouped.iloc[ind, :], num_top_venues)

districts_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Centre,Restaurant,Plaza,Mediterranean Restaurant,Café,Spanish Restaurant,Hotel,Tapas Restaurant,Gastropub,Wine Bar,Ice Cream Shop
1,Eixample,Café,Japanese Restaurant,Hotel,Bar,Spanish Restaurant,Diner,Pizza Place,Park,Plaza,Pub
2,Est,Historic Site,Garden,Mediterranean Restaurant,Scenic Lookout,Stables,Hotel,Basketball Stadium,Jazz Club,Soccer Stadium,Park
3,Mas Xirgu,Grocery Store,Restaurant,Auto Dealership,Chinese Restaurant,Café,Shopping Mall,Sporting Goods Shop,Supermarket,Electronics Store,Food & Drink Shop
4,Montjuïc,Garden,Italian Restaurant,Snack Place,Soccer Field,Wine Shop,Diner,Coffee Shop,College Cafeteria,Concert Hall,Creperie


#### 3.1.3.2 Clustering by statistical data

In [143]:
Path = 'C:/Users/user/Documents/Els_meus_documents/projectes/CompetitiveIntelligence/Data/Estadistica/Girona/'
files = os.listdir(Path)

categoriesInst = pd.read_csv(Path + files[0]).iloc[1:10, 0]
categoriesMembr = pd.read_csv(Path + files[10]).iloc[1:8, 0]
instruccio = pd.DataFrame(columns = categoriesInst)
membresLlar = pd.DataFrame(columns = categoriesMembr)

for name in files:
    barri = name.split('_')[2].split('.')[0]
    if "-" in name.split('_')[2]:
            barri = barri.split('-')[0] + " " + barri.split('-')[1]    
    if 'instruccio'in name.split('_')[0]:
        inst = pd.read_csv(Path + name)
        instruccio.loc[barri] = inst.iloc[1:10,2].tolist()
    else:
        membr = pd.read_csv(Path + name)
        membresLlar.loc[barri] = membr.iloc[1:8,2].tolist()
membresLlar.reset_index(level=0, inplace=True)
membresLlar.rename(columns = {'index' : 'Neighborhood'}, inplace = True)
instruccio.reset_index(level=0, inplace=True)
instruccio.rename(columns = {'index' : 'Neighborhood'}, inplace = True)
instruccio

Unnamed: 0,Neighborhood,No aplicable per ser menor de 16 anys,Ni llegir ni escriure,Sense estudis o educació primària incompleta,EGB. ESO. FP1 o equivalent,BUP. Batxillerat. FP2 o equivalent,Estudis universitaris de grau mig,Estudis universitaris de grau superior,Estudis superiors no universitaris,Doctorats i post-graus
0,Centre,15.13,0.43,9.44,21.96,24.7,5.82,16.48,0.86,5.18
1,Eixample,19.13,0.33,9.48,23.13,24.75,5.52,13.28,0.5,3.88
2,Est,26.25,1.75,30.23,27.96,8.26,1.22,3.17,0.16,1
3,Mas Xirgu,10.53,0,26.32,26.32,15.79,0,15.79,0,5.26
4,Montjuïc,21.27,0.21,5.13,16.85,28.35,7.01,15.79,0.25,5.13
5,Nord,20.04,1.22,21.31,30.78,16.27,2.87,5.82,0.4,1.27
6,Oest,20.52,1.07,18.46,30.55,17.91,3.31,6.51,0.21,1.46
7,Santa Eugènia,21.23,1.08,15.21,33.26,19.77,2.78,5.28,0.26,1.12
8,Sud,18.05,0.39,10.61,20.4,25.1,5.75,14.4,0.43,4.88


## 4 RESULTS

### 4.1 Clustering analysis
In this section we are going to compare the clustering results using different clustering criteria. I'm going to cluster by:
1. Venue category
2. Members in households
3. Level of education

##### 4.1.1 Clustering by venue category
The first clustering analysis is by venue gategory

In [91]:
districts_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Centre,Restaurant,Plaza,Mediterranean Restaurant,Café,Spanish Restaurant,Hotel,Tapas Restaurant,Gastropub,Wine Bar,Ice Cream Shop
1,Eixample,Café,Japanese Restaurant,Hotel,Bar,Spanish Restaurant,Diner,Pizza Place,Park,Plaza,Pub
2,Est,Historic Site,Garden,Mediterranean Restaurant,Scenic Lookout,Stables,Hotel,Basketball Stadium,Jazz Club,Soccer Stadium,Park
3,Mas Xirgu,Grocery Store,Restaurant,Auto Dealership,Chinese Restaurant,Café,Shopping Mall,Sporting Goods Shop,Supermarket,Electronics Store,Food & Drink Shop
4,Montjuïc,Garden,Italian Restaurant,Snack Place,Soccer Field,Wine Shop,Diner,Coffee Shop,College Cafeteria,Concert Hall,Creperie
5,Nord,Restaurant,Arts & Entertainment,Soccer Field,Deli / Bodega,Wine Shop,Gym,Grocery Store,Cocktail Bar,Coffee Shop,College Cafeteria
6,Oest,Hotel,Sandwich Place,Casino,Sporting Goods Shop,Plaza,Pool,Café,Burger Joint,Multiplex,Mediterranean Restaurant
7,Santa Eugènia,Restaurant,Park,Café,Bakery,Basketball Court,Mediterranean Restaurant,Pet Store,Cafeteria,Dance Studio,Soccer Field
8,Sud,Mediterranean Restaurant,Outdoors & Recreation,Athletics & Sports,Restaurant,Sporting Goods Shop,Bar,Supermarket,Soccer Field,Electronics Store,Soccer Stadium


Now I'm going to use a method of optimization of the clustring parameter *kclusters*. It is a rather manual method, consists on checking the results of the clustering with different *kclusters* values from 2 to 7. Considering that there are 9 neighborhoods I assume that using values above 7 is meaningless.

In [194]:
# set number of clusters
print("N clusters\tkluster labels")
for kclusters in range(2,8):
    Girona_grouped_clustering = Girona_grouped.drop('Neighborhood', 1)

    # run k-means clustering
    kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Girona_grouped_clustering)
    if kclusters == 3:
        kmeans_final_venues = kmeans.labels_

    # check cluster labels generated for each row in the dataframe
    print(kclusters, "\t\t", kmeans.labels_ )

N clusters	kluster labels
2 		 [1 1 1 1 0 1 1 1 1]
3 		 [1 1 1 1 0 2 1 1 1]
4 		 [1 1 1 0 3 2 1 1 1]
5 		 [2 2 4 0 1 3 2 2 2]
6 		 [2 2 4 0 1 3 2 5 2]
7 		 [4 4 5 0 3 2 4 6 1]


Looking at the results, we can see that increasing the *kclusters* does not produce a better clustering by splitting neighborhoods among clusters. Instead it looks like every time the *kclusters* value increases by one, a single neighborhood gets ascribed to the new cluster. In any case adding a new cluster makes more than one neighborhood getting included in it. Thus, we assume that adding clusters is rather irrelevant and we will choose a value of 3 for further analysis.

In [195]:
barris_venues = pd.DataFrame()
barris_venues['BARRIS'] = membresLlar['Neighborhood']
barris_venues['Code'] = kmeans_final_inst

#### 4.1.2 Clustering by statistical parameters
In this section, we make the clustering analysis for two parameters, the level of education of people in each neighborhood and the number of people living in each household. Next we show the data frames containing the data for the analysis.

There are 9 different levels of education that goes from the cateory of people under 16 which are suposed to be in the obligatory education program, to the doctorate level.

Concerning the number of people in households it ranges from 1 to 7 and above.

In [153]:
instruccio

Unnamed: 0,Neighborhood,No aplicable per ser menor de 16 anys,Ni llegir ni escriure,Sense estudis o educació primària incompleta,EGB. ESO. FP1 o equivalent,BUP. Batxillerat. FP2 o equivalent,Estudis universitaris de grau mig,Estudis universitaris de grau superior,Estudis superiors no universitaris,Doctorats i post-graus
0,Centre,15.13,0.43,9.44,21.96,24.7,5.82,16.48,0.86,5.18
1,Eixample,19.13,0.33,9.48,23.13,24.75,5.52,13.28,0.5,3.88
2,Est,26.25,1.75,30.23,27.96,8.26,1.22,3.17,0.16,1
3,Mas Xirgu,10.53,0,26.32,26.32,15.79,0,15.79,0,5.26
4,Montjuïc,21.27,0.21,5.13,16.85,28.35,7.01,15.79,0.25,5.13
5,Nord,20.04,1.22,21.31,30.78,16.27,2.87,5.82,0.4,1.27
6,Oest,20.52,1.07,18.46,30.55,17.91,3.31,6.51,0.21,1.46
7,Santa Eugènia,21.23,1.08,15.21,33.26,19.77,2.78,5.28,0.26,1.12
8,Sud,18.05,0.39,10.61,20.4,25.1,5.75,14.4,0.43,4.88


In [154]:
membresLlar

Unnamed: 0,Neighborhood,1,2,3,4,5,6,7 i més
0,Centre,38.54,27.23,15.84,11.28,4.17,1.5,1.42
1,Eixample,29.19,26.46,18.59,15.88,5.48,2.19,2.22
2,Est,22.69,22.5,18.25,18.51,10.75,4.25,3.05
3,Mas Xirgu,44.44,33.33,0,11.11,11.11,0,0
4,Montjuïc,15.07,23.74,27.76,24.87,6.4,1.86,0.31
5,Nord,30.4,24.5,17.32,15.1,6.31,3.29,3.09
6,Oest,22.91,26.98,20.87,20.11,5.71,1.73,1.7
7,Santa Eugènia,25.09,24.64,17.99,16.18,7.6,3.83,4.67
8,Sud,23.09,24.61,21.87,20.32,6.78,1.77,1.55


In [180]:
# set number of clusters
print("\t\tEducation level\t\tPeople in household\nN clusters\tkluster labels\t\tkluster labels")
for kclusters in range(2,8):
    instruccio_clustering = instruccio.drop('Neighborhood', 1)
    membresLlar_clustering = membresLlar.drop('Neighborhood', 1)

    # run k-means clustering
    kmeans_inst = KMeans(n_clusters=kclusters, random_state=0).fit(instruccio_clustering)
    kmeans_llar = KMeans(n_clusters=kclusters, random_state=0).fit(membresLlar_clustering)
    if kclusters == 3:
        kmeans_final_inst = kmeans_inst.labels_
        kmeans_final_llar = kmeans_llar.labels_

    # check cluster labels generated for each row in the dataframe
    print(kclusters, "\t\t", kmeans_inst.labels_, "\t", kmeans_llar.labels_)

		Education level		People in household
N clusters	kluster labels		kluster labels
2 		 [1 1 0 0 1 0 0 0 1] 	 [1 0 0 1 0 0 0 0 0]
3 		 [1 1 2 0 1 2 2 2 1] 	 [2 2 1 0 1 2 1 1 1]
4 		 [1 1 3 0 1 2 2 2 1] 	 [0 0 3 1 2 0 3 3 3]
5 		 [1 1 4 0 3 2 2 2 1] 	 [2 3 1 0 4 3 1 3 1]
6 		 [1 1 3 0 4 2 2 5 1] 	 [2 3 5 0 4 3 1 5 1]
7 		 [4 1 3 0 5 2 2 6 1] 	 [2 3 5 0 4 3 1 6 1]


In [181]:
kmeans_final_inst

array([1, 1, 2, 0, 1, 2, 2, 2, 1])

In [187]:
barris_instruction = pd.DataFrame()
barris_instruction['BARRIS'] = membresLlar['Neighborhood']
barris_instruction['Code'] = kmeans_final_inst
barris_household = pd.DataFrame()
barris_household['BARRIS'] = membresLlar['Neighborhood']
barris_household['Code'] = kmeans_final_llar

### 4.2 Visualization of the results
First we show the results of ploting the basic geographycal information on a map. In the following map the census sections are delimited by lines and the census districts are colored areas.

In [29]:
Barris = []
i = 0
for feature in range(0, len(Centre['features'])):
    Barris.append(Centre['features'][feature]['properties']['BARRIS'])
    i += 1
barris = pd.DataFrame(Barris, columns = ['BARRIS'])
barris['Code'] =  range(0, len(Barris))
barris.loc[barris['BARRIS'] == 'Eixample', ['Code']] = 3
barris.loc[barris['BARRIS'] == 'Mas Xirgu', ['Code']] = 6

In [191]:
def drawBasicMap(JSON1, JSON2, barris):
    lat = 41.9802474
    long = 2.8236477
    gir_map = folium.Map(location=[lat,long], zoom_start=13)

    # for i in range(0,len(labels)):
    #     folium.Marker([Labels[i][1], Labels[i][2]],
    #                  popup=Labels[i][0]).add_to(gir_map)

    folium.Choropleth(
        geo_data = JSON1,
        fill_opacity = 0.005, 
        line_opacity = 1
    ).add_to(gir_map)

    choropleth = folium.Choropleth(
        geo_data = JSON2,
        data = barris,
        columns = ['BARRIS', 'Code'],
        key_on = 'feature.properties.BARRIS',
        fill_color = 'Dark2', 
        fill_opacity = 0.5, 
        line_opacity = 0
    ).add_to(gir_map)

    # add labels indicating the name of the neighborhood
    style_function = "font-size: 15px; font-weight: bold"
    choropleth.geojson.add_child(
        folium.features.GeoJsonTooltip(['BARRIS'], style=style_function, labels=False))

    return gir_map
def drawMap(JSON, barris):
    lat = 41.9802474
    long = 2.8236477
    gir_map = folium.Map(location=[lat,long], zoom_start=13)

    choropleth = folium.Choropleth(
        geo_data = JSON,
        data = barris,
        columns = ['BARRIS', 'Code'],
        key_on = 'feature.properties.BARRIS',
        fill_color = 'Dark2', 
        fill_opacity = 0.5, 
        line_opacity = 1
    ).add_to(gir_map)

    # add labels indicating the name of the neighborhood
    style_function = "font-size: 15px; font-weight: bold"
    choropleth.geojson.add_child(
        folium.features.GeoJsonTooltip(['BARRIS'], style=style_function, labels=False))

    return gir_map

Next we are going to draw the map of Girona with the census sections delimited by a line, and the census districts or neighborhoods shown as colored areas. Remember that the census sections have been used as a unit to retrieve Foursquare information, but the rest of the analysis are going to be made with neighborhood areas.

In [178]:
drawBasicMap(Girona, Centre, barris, )

In [197]:
drawMap(Centre, barris_venues)

In [192]:
drawMap(Centre, barris_instruction)

In [193]:
drawMap(Centre, barris_household)

Next we plot the four maps in a combined image to better compare them
<img src="Imatges/FinalResults.png" width="1200" />

**Discussion of the results**
The results are interesting because they show a perfect match between the venues landscape and the education landscape. Both clustering criteria yield the same result in terms of clustering of neighborhoods. This might indicate not only a correlation but a potential causal correlation between the two variables. Higher educated people tend to live in central parts of the city, and the profile of business that are stablished in these neighborhoods is fitting the profile of the people living there.

Instead, the relationship between venues and number of people in households is not fitting very much. Here we see a different pattern likely due to the fact that families clerly live in the perifery, but also in central neighborhoods but not in downtown.

Finally, all thre clustering criteria show that there is a spacial neighborood, the one which is called Mas Xirgu and that appear in green in the three clustering maps. It is a neighborhood ocupied in great extent by an industrial area.

## 5 Discussion
This preliminary work that has been done comparing the results of clustering of neighborhoods using different clustering criteria shows its potential as a method to deliver relevant insights to stakeholders.
* Companies whishing to open their business in Girona might look at the most common venues in each neighborhood and then compare its socioeconomic landscape with their target audiences to make their choices. For instance, if they target families with a good level of education, then Montjuïc and Sud might be two of the neighborhoods to choose.
* On the other hand, if the city council wants to change the venues landscape of neighborhoods with families with a relatively low level of education, then they might make some plans for the neighborhopds Est and Oest.

## 6 Conclusions
1. More than 80% of the time consumed in the project was devoted to get and manipulate the data in order to have it right for the analysis.
2. Using just one type of data from Foursquare and two types of socioeconomic data we got some interesting insights, but there are many further analysis that could be done within the same framework.